# Pre Embedding the News with BERT

In [2]:
BERT_model= 'paraphrase-MiniLM-L6-v2' #'all-mpnet-base-v2'

In [14]:
import os
save_dir='./News-embedding-'+BERT_model
if not os.path.exists(save_dir):
    os.makedirs(save_dir)


## Initilize the Embedding model

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(BERT_model).to('cuda:0')

In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

## Read the news data with seperation symbol as |

In [6]:
df=pd.read_csv('./toydata_with_text.csv')

In [7]:
df.head()

,date,Channel 1,text
0,1800-01-01 00:00:00,0.000000,The channel 1 period is 84 and goes steady
1,1800-01-01 00:01:00,0.074730,The channel 1 period is 84 and goes steady
2,1800-01-01 00:02:00,0.149042,The channel 1 period is 84 and goes steady
3,1800-01-01 00:03:00,0.222521,The channel 1 period is 84 and goes steady
4,1800-01-01 00:04:00,0.294755,The channel 1 period is 84 and goes steady


In [8]:
# get the Date range of the news
df['date'] = pd.to_datetime(df['date'])
df['date'].min(), df['date'].max()

(Timestamp('1800-01-01 00:00:00'), Timestamp('1800-05-17 04:31:00'))

## Get the embedding news in daily basis and save as individual file

In [22]:
empty_date=[]
# use tqdm to show the progress

progress_bar = tqdm(pd.date_range(start=df['date'].min(), end=df['date'].max(),freq='min'), desc="Processing Dates")
days=[]
for day in progress_bar:
    if len(days)<499:
        days.append(day)
        # print(len(days))
        continue
    
    days.append(day)
    # get the news of the day
    # news = list(df[df['date'] == days]['text'])
    news = list(df.loc[df['date'].isin(days)]['text'])
    # get the embedding of the news
    news_embedding = model.encode(news)
    # normalize the embedding
    news_embedding = news_embedding / np.linalg.norm(news_embedding, axis=1, keepdims=True)
    for d in range(len(days)):
        # save the embedding
        np.save(os.path.join(save_dir,'News-'+str(days[d])+'.npy'), news_embedding[d:d+1,:])
        progress_bar.set_postfix(CurrentDate=str(day)+' News='+str(len(news)))
    days=[]

Processing Dates: 100%|██████████| 196112/196112 [19:48<00:00, 165.03it/s, CurrentDate=1800-05-17 02:39:00 News=500]


## Check empty date

In [9]:
len(empty_date)

642